In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from model import Discriminator,Generator,initialize_weights

torch.Size([10, 1, 1, 1])
torch.Size([10, 3, 64, 64])


In [3]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [ ]:
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
Z_DIMS = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64 
FEATURES_GEN = 64
